# Hero Overall Win Rates

Simple calculation of the overall win rate for each hero over a series of database files. Edit DB_FILES list to contain list of files.

In [ ]:
import sqlite3
import os
import meta
import json
import pandas as pd
import datetime as dt
import re
import plotly.express as px
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import numpy as np

## Merge databases and remove duplicates

In [ ]:
DB_FILES=[]
for filename in os.listdir("."):    
    if re.match("^.*db$",filename) is not None:    
        DB_FILES.append(filename)
                    
rows=[]
for this_file in DB_FILES:
    conn=sqlite3.connect(this_file)
    c=conn.cursor()
    c.execute("SELECT match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent FROM dota_stats")
    rows.extend(c.fetchall())
    print("{0} {1}".format(this_file,len(rows)))

In [ ]:
rows=list(set(rows))
print("Removing duplicates: {0}".format(len(rows)))

In [ ]:
import mariadb

conn_maria = mariadb.connect(
    user=os.environ['DOTA_USERNAME'],
    password=os.environ['DOTA_PASSWORD'],
    host="localhost",
    database="dota")

c=conn_maria.cursor()
c.execute("select count(*) from dota_matches")
print("Rows before merge: {0} ".format(c.fetchall()[0][0]))

# Batch samples in
BATCH_SIZE=2000
for i in range(len(rows)//BATCH_SIZE+1):    
    begin=i*BATCH_SIZE
    end=min(len(rows),(i+1)*BATCH_SIZE)
    print("{0} to {1}".format(begin,end))
    #c.executemany('INSERT IGNORE INTO dota_matches (match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent) values(%d, %d, %s, %s, %d, %d, %s, %s)', 
    #              list(rows)[begin:end])
    c.executemany('INSERT IGNORE INTO dota_matches (match_id, start_time, radiant_heroes, dire_heroes, radiant_win, api_skill, items, gold_spent) values(?, ?, ?, ?, ?, ?, ?, ?)', 
                  list(rows)[begin:end])
    
conn_maria.commit()
c.execute("select count(*) from dota_matches")
print("Rows after merge: {0} ".format(c.fetchall()[0][0]))
conn_maria.close()